In [1]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark import SparkContext

# 전역 SparkContext 정리
SparkContext.setSystemProperty('spark.driver.host', '127.0.0.1')
SparkContext.setSystemProperty('spark.driver.bindAddress', '127.0.0.1')

# 모든 활성 context 중지
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# 새로 생성
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.driver.host", "127.0.0.1") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/16 19:06:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Question 1 

In [2]:
spark.version

'4.1.1'

# Question 2 

In [28]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [9]:
df = df.repartition(4)

In [10]:
df.write.parquet('data/pq/yellow/2024/04/')

In [15]:
!ls -lh data/pq/yellow/2024/04

total 188928
-rw-r--r--  1 doyeongyu  staff     0B  2 16 19:17 _SUCCESS
-rw-r--r--  1 doyeongyu  staff    22M  2 16 19:17 part-00000-f7b84d8b-7fa5-413e-8f95-839de17f9e63-c000.snappy.parquet
-rw-r--r--  1 doyeongyu  staff    22M  2 16 19:17 part-00001-f7b84d8b-7fa5-413e-8f95-839de17f9e63-c000.snappy.parquet
-rw-r--r--  1 doyeongyu  staff    22M  2 16 19:17 part-00002-f7b84d8b-7fa5-413e-8f95-839de17f9e63-c000.snappy.parquet
-rw-r--r--  1 doyeongyu  staff    22M  2 16 19:17 part-00003-f7b84d8b-7fa5-413e-8f95-839de17f9e63-c000.snappy.parquet


# Question 3 

In [16]:
from pyspark.sql import functions as F

In [32]:
df = spark.read.parquet('data/pq/yellow/2024/04/')

In [33]:
df.registerTempTable('yellow')

In [40]:
spark.sql("""
    SELECT COUNT(*)
    FROM yellow
    WHERE TO_DATE(tpep_pickup_datetime) = '2024-10-15'
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [36]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-04 23:19:27|  2024-10-04 23:43:28|              1|         6.07|         1|                 N|         229|         231|           1|       31.0|  1.0|    0.5|       4.

# Question 4

In [41]:
spark.sql("""
    SELECT 
        MAX((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600) as max_trip_hours
    FROM yellow
""").show()

+------------------+
|    max_trip_hours|
+------------------+
|162.61777777777777|
+------------------+



# Question 5

Spark's Application UI runs on port 4040 by default.

# Question 6

In [42]:
df_zones = spark.read \
            .option("header", "true") \
            .csv('taxi_zone_lookup.csv')

In [45]:
df_zones.registerTempTable('zones')

In [46]:
spark.sql("""
    SELECT 
        z.Zone,
        COUNT(*) as pickup_count
    FROM yellow y
    JOIN zones z ON y.PULocationID = z.LocationID
    GROUP BY z.Zone
    ORDER BY pickup_count ASC
    LIMIT 10
""").show()

+--------------------+------------+
|                Zone|pickup_count|
+--------------------+------------+
|Governor's Island...|           1|
|       Rikers Island|           2|
|       Arden Heights|           2|
|         Jamaica Bay|           3|
| Green-Wood Cemetery|           3|
|Charleston/Totten...|           4|
|   Rossville/Woodrow|           4|
|       West Brighton|           4|
|       Port Richmond|           4|
|Eltingville/Annad...|           4|
+--------------------+------------+

